In [16]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [99]:
import json
import pandas as pd
import os
import numpy as np
import distance

In [6]:
data = {}
clean_data = {}

for f in os.listdir('/home/katya/start/HIS3InterspeciesEpistasis/Data/'):
    if 'csv' in f:
        data[f[:-16]] = pd.DataFrame.from_csv('/home/katya/start/HIS3InterspeciesEpistasis/Data/' + f, sep='\t')
        median = np.median(data[f[:-16]].index.str.len())
        
        clean_data[f[:-16]] = data[f[:-16]][(data[f[:-16]].nonsense == 0) & (data[f[:-16]].middle == 1) & \
                                            (data[f[:-16]].nat_lib == 1) & (data[f[:-16]].stop == 0) & \
                                            (data[f[:-16]].index.str.len() == median)]

In [8]:
source = ['23','23','23','24','24','40','40','41','41','25','25','26','26','27','27','28','28','34','34','35','35','36','36','37','37','38','38','39','39','29','29','30','30','31','31','32','32','33','33']
target = ['Ylip','Anid','24','40','25','Sjap','41','Soct','Spom','Ncra','26','Agos','27','Cgla','28','29','34','Cgui','35','36','37','Clus','Dhan','38','39','Calb','Ctro','Cpar','Lelo','30','32','Scas','31','Scer','Sbay','Klac','33','Kwal','Sklu']

nodes = list(source)
nodes.extend(target)
nodes = list(set(nodes))

In [101]:
location = {}
for node in nodes:
    location[node] = {}
    location[node]['x'] = np.random.randint(1000)
    location[node]['y'] = np.random.randint(1000)

In [104]:
location

{'23': {'x': 95, 'y': 234},
 '24': {'x': 450, 'y': 951},
 '25': {'x': 866, 'y': 934},
 '26': {'x': 32, 'y': 367},
 '27': {'x': 527, 'y': 150},
 '28': {'x': 189, 'y': 125},
 '29': {'x': 35, 'y': 940},
 '30': {'x': 17, 'y': 69},
 '31': {'x': 150, 'y': 622},
 '32': {'x': 297, 'y': 695},
 '33': {'x': 750, 'y': 539},
 '34': {'x': 569, 'y': 36},
 '35': {'x': 314, 'y': 544},
 '36': {'x': 645, 'y': 520},
 '37': {'x': 400, 'y': 539},
 '38': {'x': 946, 'y': 673},
 '39': {'x': 342, 'y': 384},
 '40': {'x': 702, 'y': 889},
 '41': {'x': 702, 'y': 733},
 'Agos': {'x': 655, 'y': 537},
 'Anid': {'x': 38, 'y': 316},
 'Calb': {'x': 618, 'y': 858},
 'Cgla': {'x': 392, 'y': 180},
 'Cgui': {'x': 503, 'y': 893},
 'Clus': {'x': 753, 'y': 31},
 'Cpar': {'x': 77, 'y': 847},
 'Ctro': {'x': 986, 'y': 730},
 'Dhan': {'x': 194, 'y': 93},
 'Klac': {'x': 979, 'y': 206},
 'Kwal': {'x': 842, 'y': 806},
 'Lelo': {'x': 310, 'y': 919},
 'Ncra': {'x': 736, 'y': 245},
 'Sbay': {'x': 947, 'y': 144},
 'Scas': {'x': 652, 'y': 

In [102]:
chunks = ['S'+str(i) for i in range(1,13)]

In [106]:
for chunk in chunks:
    print chunk
    data = {}
    data['nodes'] = []
    for node in nodes:

            if node.isdigit():
                if len(clean_data[chunk][clean_data[chunk]['dist_node_'+node]==0])!=0:
                    fitness = float(clean_data[chunk][clean_data[chunk]['dist_node_'+node]==0].s)
                    data['nodes'].append({'id':node, 'fitness':fitness, 'x': location[node]['x'], 
                                          'y': location[node]['y'], 'group':0})

                else:
                    fitness = 0.45
                    data['nodes'].append({'id':node, 'fitness':fitness, 'x': location[node]['x'], 
                                          'y': location[node]['y'], 'group':2})
                    
            else:
                
                if len(clean_data[chunk][clean_data[chunk]['dist_'+node]==0])!=0:
                    fitness = float(clean_data[chunk][clean_data[chunk]['dist_'+node]==0].s)
                    data['nodes'].append({'id':node, 'fitness':fitness, 'x': location[node]['x'], 
                                          'y': location[node]['y'],  'group':0})

                else:
                    fitness = 0.45
                    data['nodes'].append({'id':node, 'fitness':fitness, 'x': location[node]['x'], 
                                          'y': location[node]['y'], 'group':2})
            
    data['links'] = []

    for i in range(len(source)):
        data['links'].append({'source':source[i], 'target':target[i], 'group':0})

######################################################################################################################        
        
    tempNodes = []
    tempNodes2 = []

    for node in nodes:
        if node.isdigit():
            nodeList = clean_data[chunk][clean_data[chunk]['dist_node_'+node]==1].index
            for sq1 in nodeList:
                if sq1+node+'1' not in tempNodes:
                    tempNodes.append(sq1+node+'1')
                    data['nodes'].append({'id':sq1+node+'1', 'fitness':clean_data[chunk].ix[sq1].s, 'group':1})
                data['links'].append({'source':node, 'target':sq1+node+'1', 'group':1})
                
                nodeList2 = clean_data[chunk][clean_data[chunk]['dist_node_'+node]==2].index
                for sq2 in nodeList2:
                    if distance.hamming(sq1,sq2) == 1:
                        if sq2+'_'+sq1+node+'2' not in tempNodes2:
                            tempNodes2.append(sq2+'_'+sq1+node+'2')
                            data['nodes'].append({'id':sq2+'_'+sq1+node+'2', 
                                                  'fitness':clean_data[chunk].ix[sq2].s, 'group':1})
                        data['links'].append({'source':sq1+node+'1', 'target':sq2+'_'+sq1+node+'2', 'group':1})

        else:
            nodeList = clean_data[chunk][clean_data[chunk]['dist_'+node]==1].index
            for sq1 in nodeList:
                if sq1+node+'1' not in tempNodes:
                    tempNodes.append(sq1+node+'1')
                    data['nodes'].append({'id':sq1+node+'1', 'fitness':clean_data[chunk].ix[sq1].s, 'group':1})
                data['links'].append({'source':node, 'target':sq1+node+'1', 'group':1})
                
    with open('/home/katya/start/HIS3InterspeciesEpistasis/Analysis/Katya/Fig3/' + chunk + '.json', 'w+') as outfile:  
        json.dump(data, outfile)

S1
S2
S3
S4
S5
S6
S7
S8
S9
S10
S11
S12
